# PLC Knowledge Graph ChatBot (Gemini + RDFLib) – Text2SPARQL Template

Dieses Notebook ist ein **Template**, um Fragen zu einem PLC-Programm über einen **Wissensgraphen (TTL/RDF)** zu beantworten.

**Modus: Text2SPARQL (schnell)**
1) LLM erzeugt eine SPARQL SELECT Query  
2) RDFLib führt sie auf dem lokalen Graph aus  
3) Ergebnis wird als **Tabelle (Pandas DataFrame)** angezeigt

✅ Anforderungen:
- Gemini API Key aus Datei laden
- Text2SPARQL ausführen
- Tabellen bleiben Tabellen (kein eingebetteter Fließtext)
- Ausgabe zeigt **nur** die ausgeführte SPARQL Query (und optional die Tabelle, wenn vorhanden)


In [21]:
# (1) Install (einmalig)
!pip install -U rdflib pandas langchain-core langchain-openai


   ---------------------------------------- 0.0/878.7 kB ? eta -:--:--
   ---------------------------------------- 878.7/878.7 kB 13.1 MB/s  0:00:00

   ---------------------------------------- 0/3 [regex]
   ------------- -------------------------- 1/3 [tiktoken]
   -------------------------- ------------- 2/3 [langchain-openai]
   -------------------------- ------------- 2/3 [langchain-openai]
   -------------------------- ------------- 2/3 [langchain-openai]
   ---------------------------------------- 3/3 [langchain-openai]



## (2) Konfiguration: Gemini API Key aus Datei laden

Du hast den Key in:
`C:\Users\Alexander Verkhov\Desktop\Gemini API Key.txt`

Die LangChain Integration liest standardmäßig `GOOGLE_API_KEY`.


In [1]:
# === ZELLE 2: KONFIGURATION ===
import os
from pathlib import Path

# === Pfad zur TTL-Datei (bleibt gleich) ===
TTL_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test2_filled.ttl"

# === OpenAI API Key aus Datei laden ===
# Pfad zu deiner OpenAI Key Textdatei
openai_key_path = r"C:\Users\Alexander Verkhov\Desktop\OpenAI API Key.txt"

try:
    # Key einlesen und Leerzeichen entfernen
    key = Path(openai_key_path).read_text(encoding="utf-8").strip()
    
    # OpenAI benötigt die Variable OPENAI_API_KEY
    os.environ["OPENAI_API_KEY"] = key
    print("✅ OPENAI_API_KEY erfolgreich gesetzt.")
    
except Exception as e:
    print(f"❌ Fehler beim Laden des Keys: {e}")

✅ OPENAI_API_KEY erfolgreich gesetzt.


## (3) Wissensgraph laden (TTL)

In [2]:
from rdflib import Graph

g = Graph()
g.parse(TTL_PATH, format="turtle")

print("✅ Graph geladen.")
print("Triples:", len(g))
print("Namespaces (Auszug):", list(g.namespaces())[:8])


✅ Graph geladen.
Triples: 16410
Namespaces (Auszug): [('brick', rdflib.term.URIRef('https://brickschema.org/schema/Brick#')), ('csvw', rdflib.term.URIRef('http://www.w3.org/ns/csvw#')), ('dc', rdflib.term.URIRef('http://purl.org/dc/elements/1.1/')), ('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#')), ('dcmitype', rdflib.term.URIRef('http://purl.org/dc/dcmitype/')), ('dcterms', rdflib.term.URIRef('http://purl.org/dc/terms/')), ('dcam', rdflib.term.URIRef('http://purl.org/dc/dcam/')), ('doap', rdflib.term.URIRef('http://usefulinc.com/ns/doap#'))]


In [5]:
# === ZELLE 3 KOMPLETT ERSETZEN (Python-Loop statt SPARQL) ===

from rdflib import Graph, RDF

def shorten_uri(uri, graph):
    u = str(uri)
    # Manuelle Verkürzung für sauberen Prompt
    u = u.replace("http://www.semanticweb.org/AgentProgramParams/dp_", "dp:")
    u = u.replace("http://www.semanticweb.org/AgentProgramParams/op_", "op:")
    u = u.replace("http://www.semanticweb.org/AgentProgramParams/class_", "ag:class_") # Explizit class_ Prefix
    u = u.replace("http://www.semanticweb.org/AgentProgramParams/", "ag:")
    
    # Fallback
    try:
        return graph.namespace_manager.normalizeUri(uri)
    except:
        return u

def get_kg_schema_fast(graph):
    # 1. Cache aufbauen: Welches Subjekt hat welchen Typ?
    # Wir speichern nur Typen, die "class_" im Namen haben.
    entity_types = {}
    
    # Iteriere über alle Typ-Zuweisungen (s a type)
    for s, _, t in graph.triples((None, RDF.type, None)):
        t_str = str(t)
        if "class_" in t_str:
            # Wir nehmen den ersten gefundenen Typ (falls mehrere da sind, reicht einer fürs Schema)
            entity_types[s] = shorten_uri(t, graph)

    # Liste der Klassen für den Prompt
    unique_classes = sorted(list(set(entity_types.values())))
    
    # 2. Beziehungen (op_) und Attribute (dp_) scannen
    relations = set()
    attributes = set()
    
    # Iteriere über ALLE Tripel im Graph (sehr schnell in Python)
    for s, p, o in graph:
        p_str = str(p)
        
        # Fall A: Object Property (op_)
        if "op_" in p_str and s in entity_types:
            # Für Relations brauchen wir auch den Typ des Objekts
            if o in entity_types:
                s_type = entity_types[s]
                o_type = entity_types[o]
                p_short = shorten_uri(p, graph)
                relations.add(f"{s_type} --[{p_short}]--> {o_type}")
                
        # Fall B: Data Property (dp_)
        elif "dp_" in p_str and s in entity_types:
            s_type = entity_types[s]
            p_short = shorten_uri(p, graph)
            attributes.add((s_type, p_short))

    # --- Ausgabe zusammenbauen ---
    schema_parts = []
    
    schema_parts.append(f"### KLASSEN:\n" + ", ".join(unique_classes))
    
    schema_parts.append("\n### BEZIEHUNGEN (Struktur):")
    schema_parts.append("Format: SubjektKlasse --[Relation]--> ObjektKlasse")
    schema_parts.append("\n".join(sorted(list(relations))))
    
    schema_parts.append("\n### ATTRIBUTE (Daten):")
    # Attribute gruppieren
    attr_dict = {}
    for c, p in attributes:
        if c not in attr_dict: attr_dict[c] = []
        attr_dict[c].append(p)
        
    attr_lines = []
    for c in sorted(attr_dict.keys()):
        attr_lines.append(f"\n{c}:")
        for p in sorted(attr_dict[c]):
             attr_lines.append(f"  - {p}")
             
    schema_parts.append("\n".join(attr_lines))

    return "\n".join(schema_parts)

# Ausführen
kg_schema_context = get_kg_schema_fast(g)

print("--- Extrahiertes Schema (Fast Python Version) ---")
print(kg_schema_context)

--- Extrahiertes Schema (Fast Python Version) ---
### KLASSEN:
ag:class_FBInstance, ag:class_FBType, ag:class_GlobalVariableList, ag:class_IOChannel, ag:class_PLCProject, ag:class_POUCall, ag:class_ParameterAssignment, ag:class_Port, ag:class_Program, ag:class_SignalSource, ag:class_StandardFBType, ag:class_Variable

### BEZIEHUNGEN (Struktur):
Format: SubjektKlasse --[Relation]--> ObjektKlasse
ag:class_FBInstance --[op:isInstanceOfFBType]--> ag:class_FBType
ag:class_FBInstance --[op:isInstanceOfFBType]--> ag:class_StandardFBType
ag:class_FBType --[op:containsPOUCall]--> ag:class_POUCall
ag:class_FBType --[op:hasInternalVariable]--> ag:class_Variable
ag:class_FBType --[op:hasPort]--> ag:class_Port
ag:class_FBType --[op:usesVariable]--> ag:class_Variable
ag:class_GlobalVariableList --[op:listsGlobalVariable]--> ag:class_Variable
ag:class_PLCProject --[op:consistsOfPOU]--> ag:class_FBType
ag:class_PLCProject --[op:consistsOfPOU]--> ag:class_Program
ag:class_POUCall --[op:callsPOU]--> ag:

## (4) SPARQL Helper (RDFLib)

- Nur **SELECT** erlauben  
- `LIMIT` automatisch ergänzen  
- Ergebnis als JSON-Objekt


In [6]:
import json
from typing import Any, Dict

DEFAULT_PREFIXES = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ag:  <http://www.semanticweb.org/AgentProgramParams/>
PREFIX dp:  <http://www.semanticweb.org/AgentProgramParams/dp_>
PREFIX op:  <http://www.semanticweb.org/AgentProgramParams/op_>
"""


def sparql_select_raw(query: str, max_rows: int = 50) -> Dict[str, Any]:
    q = query.strip()

    # Prefixes auto-inject, falls nicht vorhanden
    if "PREFIX" not in q.upper():
        q = DEFAULT_PREFIXES + "\n" + q

    q_lc = q.lower().lstrip()
    if "select" not in q_lc:
        return {"ok": False, "error": "Query must contain SELECT.", "query": q}

    # Minimal Safety: keine Updates
    if "insert" in q_lc or "delete" in q_lc or "update" in q_lc:
        return {"ok": False, "error": "Only SELECT queries are allowed.", "query": q}

    if "limit" not in q_lc:
        q += f"\nLIMIT {max_rows}"

    try:
        res = g.query(q)
        rows = [dict(r.asdict()) for r in res]
        rows = [{k: (str(v) if v is not None else None) for k, v in row.items()} for row in rows]
        return {"ok": True, "row_count": len(rows), "rows": rows, "query": q}
    except Exception as e:
        return {"ok": False, "error": str(e), "query": q}


## (5) Text2SPARQL (Gemini)

Hier erzeugt Gemini **nur** die SPARQL SELECT Query.

Wichtig:
- Das LLM soll **nichts erklären**
- Es soll **nur** die Query zurückgeben


In [7]:
# === ZELLE 5 KOMPLETT ERSETZEN (Mit Hierarchie-Optimierung) ===

from langchain_openai import ChatOpenAI

t2s_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=1024,
)

# Hier injizieren wir das dynamische Schema
TEXT2SPARQL_SYSTEM = f"""Du bist ein Text2SPARQL Generator für einen PLC Knowledge Graph (RDF).
Deine Aufgabe: Wandle die Frage des Users in eine SPARQL SELECT Query um.

### SCHEMA INFO (Verfügbare Klassen & Properties):
{kg_schema_context}

### WICHTIGES DOMAIN-WISSEN:
- "POU" (Program Organization Unit) ist der Oberbegriff für **Programme** (ag:class_Program), **Funktionsbausteine** (ag:class_FBType) und Funktionen.
- Wenn nach "POUs" gefragt wird, suche nach ALLEM, was unter `ag:class_POU` fällt.

### REGELN:
1. Antworte AUSSCHLIESSLICH mit dem SPARQL-Code.
2. Nutze IMMER diese Prefixes:
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX ag: <http://www.semanticweb.org/AgentProgramParams/>
   PREFIX dp: <http://www.semanticweb.org/AgentProgramParams/dp_>
   PREFIX op: <http://www.semanticweb.org/AgentProgramParams/op_>

3. **HIERARCHIEN BEACHTEN**:
   Wenn nach einer Oberklasse gefragt wird (z.B. POU), nutze den Pfad `a/rdfs:subClassOf*`, um auch alle Unterklassen (Programme, FBs) zu finden.
   
   Falsch (findet nur Programme):
   `?s a ag:class_Program .`
   
   Richtig (findet alles, was ein POU ist):
   `?s a/rdfs:subClassOf* ag:class_POU .`

4. Beispiel-Query (Alle POUs finden):
   SELECT ?name ?type WHERE {{
     ?s a/rdfs:subClassOf* ag:class_POU .
     ?s dp:hasPOUName ?name .
     ?s a ?type .
   }}
"""

def text2sparql(question: str) -> str:
    msg = [
        ("system", TEXT2SPARQL_SYSTEM),
        ("user", question),
    ]
    try:
        response = t2s_llm.invoke(msg)
        content = response.content
        
        final_text = str(content).strip()
        final_text = final_text.replace("```sparql", "").replace("```", "").strip()
        
        return final_text
        
    except Exception as e:
        return f"Error calling OpenAI: {str(e)}"

## (6) Ausführen: Query drucken + Tabelle anzeigen

Ausgabe:
1) **nur** die SPARQL Query (print)  
2) **wenn vorhanden**: Tabelle als DataFrame (keine weitere Erklärung)

Wenn keine Zeilen gefunden werden: nur Query wird gedruckt.


In [ ]:
# === ZELLE 6 / 16 (Ausführung) ===
import pandas as pd
from IPython.display import display

# === Pandas Optionen für vollständige Anzeige ===
pd.set_option('display.max_rows', None)      # Alle Zeilen anzeigen
pd.set_option('display.max_columns', None)   # Alle Spalten anzeigen
pd.set_option('display.max_colwidth', None)  # Inhalt in Zellen nicht abschneiden
pd.set_option('display.width', 1000)         # Breite der Ausgabe erhöhen

def run_text2sparql(question: str, max_rows: int = 50):
    print(f"❓ Frage: {question}\n")
    
    # 1. Query generieren
    query_raw = text2sparql(question)
    
    # 2. Bereinigung: Markdown-Code-Blöcke entfernen
    # Manche Modelle liefern ```sparql ... ```
    query = query_raw.replace("```sparql", "").replace("```", "").strip()

    print("🔍 Generierte SPARQL Query:")
    print("-" * 40)
    print(query)
    print("-" * 40 + "\n")

    # 3. Ausführen
    result = sparql_select_raw(query, max_rows=max_rows)
    
    if not result.get("ok"):
        print(f"❌ Fehler bei der Ausführung: {result.get('error')}")
        # Debugging: Zeige was schief lief, falls die Query komisch aussieht
        # print(f"Query war: {query}")
        return None

    if result.get("row_count", 0) == 0:
        print("⚠️ Keine Ergebnisse gefunden.")
        return None

    # 4. Ergebnis anzeigen
    df = pd.DataFrame(result["rows"])
    print(f"✅ {len(df)} Ergebnisse gefunden:")
    display(df)
    return df

# Test:
run_text2sparql("Welche POUs ruft 'HRL_SkillSet' auf?")

❓ Frage: Welche POUs ruft 'HRL_SkillSet' auf?

🔍 Generierte SPARQL Query:
----------------------------------------
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ag: <http://www.semanticweb.org/AgentProgramParams/>
PREFIX dp: <http://www.semanticweb.org/AgentProgramParams/dp_>
PREFIX op: <http://www.semanticweb.org/AgentProgramParams/op_>

SELECT ?pouName ?pouType WHERE {
  ?pouCall op:hasCallerVariable ?variable .
  ?variable dp:hasVariableName "HRL_SkillSet" .
  ?pouCall op:callsPOU ?pou .
  ?pou a/rdfs:subClassOf* ag:class_POU .
  ?pou dp:hasPOUName ?pouName .
  ?pou a ?pouType .
}
----------------------------------------

⚠️ Keine Ergebnisse gefunden.


Test Langgraph

In [10]:
# === ZELLE 7 / 16 (LangChain GraphSparqlQAChain) ===

# Falls noch nicht installiert:
try:
    from langchain_community.chains.graph_qa.sparql import GraphSparqlQAChain
    from langchain_community.graphs import RdfGraph
except ImportError:
    !pip install -U langchain langchain-community
    from langchain_community.chains.graph_qa.sparql import GraphSparqlQAChain
    from langchain_community.graphs import RdfGraph



In [12]:
# === FIX ZELLE: GraphSparqlQAChain ohne Backticks + Debug/Steps ===

from langchain_community.chains.graph_qa.sparql import GraphSparqlQAChain
from langchain_community.graphs import RdfGraph

# Wichtig: entfernt ```sparql ... ``` oder <sparql>...</sparql>
from langchain_community.chains.graph_qa.neptune_sparql import extract_sparql

# Debug / Zwischensteps
from langchain_core.callbacks.stdout import StdOutCallbackHandler
from langchain_core.globals import set_debug, set_verbose

set_debug(True)
set_verbose(True)

# 1) RDF Graph laden
lc_graph = RdfGraph(
    source_file=TTL_PATH,
    serialization="ttl",
    standard="rdf",
)
lc_graph.load_schema()

# 2) Monkey-Patch: vor jeder Query automatisch Code-Fences entfernen
_orig_query = lc_graph.query

def _clean_query(q: str):
    q_clean = extract_sparql(q)
    return _orig_query(q_clean)

lc_graph.query = _clean_query

# 3) Chain bauen (verbose=True zeigt SPARQL + Context-Ausgabe)
graph_chain = GraphSparqlQAChain.from_llm(
    llm=llm_for_chain,
    graph=lc_graph,
    return_sparql_query=True,
    allow_dangerous_requests=True,
    verbose=True,
)

# 4) Call mit Callback (zeigt Prompts/Outputs live)
question = "Welche POUs ruft 'HRL_SkillSet' auf?"

out = graph_chain.invoke(
    {"query": question},
    config={"callbacks": [StdOutCallbackHandler()]}
)

print("\n--- SPARQL (raw) ---")
print(out.get("sparql_query", ""))

print("\n--- SPARQL (clean) ---")
print(extract_sparql(out.get("sparql_query", "")))

print("\n--- Antwort ---")
print(out.get("result", ""))




> Entering new GraphSparqlQAChain chain...
[chain/start] [chain:GraphSparqlQAChain] Entering Chain run with input:
{
  "query": "Welche POUs ruft 'HRL_SkillSet' auf?"
}


> Entering new LLMChain chain...
[chain/start] [chain:GraphSparqlQAChain > chain:LLMChain] Entering Chain run with input:
{
  "prompt": "Welche POUs ruft 'HRL_SkillSet' auf?"
}
Prompt after formatting:
Task: Identify the intent of a prompt and return the appropriate SPARQL query type.
You are an assistant that distinguishes different types of prompts and returns the corresponding SPARQL query types.
Consider only the following query types:
* SELECT: this query type corresponds to questions
* UPDATE: this query type corresponds to all requests for deleting, inserting, or changing triples
Note: Be as concise as possible.
Do not include any explanations or apologies in your responses.
Do not respond to any questions that ask for anything else than for you to identify a SPARQL query type.
Do not include any unnecessary 

In [11]:
import pandas as pd
from IPython.display import display

# 1) RDF Graph Wrapper (lädt die gleiche TTL wie du bereits nutzt)
lc_graph = RdfGraph(
    source_file=TTL_PATH,
    serialization="ttl",
    standard="rdf",   # alternativ: "owl" wenn du OWL-lastige Ontologien hast
)

# Optional: Schema laden (kann helfen, wenn get_schema sonst leer bleibt)
lc_graph.load_schema()

# 2) LLM wiederverwenden (falls t2s_llm schon existiert)
try:
    llm_for_chain = t2s_llm
except NameError:
    from langchain_openai import ChatOpenAI
    llm_for_chain = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        max_tokens=1024,
    )

# 3) GraphSparqlQAChain bauen
graph_chain = GraphSparqlQAChain.from_llm(
    llm=llm_for_chain,
    graph=lc_graph,
    return_sparql_query=True,         # damit du die generierte Query bekommst
    allow_dangerous_requests=True,    # required Opt-In (LangChain Security)
    verbose=False,
)

def run_graphsparql_chain(question: str, max_rows: int = 30, show_llm_answer: bool = False):
    """
    1) erzeugt SPARQL via GraphSparqlQAChain
    2) druckt SPARQL
    3) führt SPARQL mit deiner vorhandenen sparql_select_raw(...) aus
    4) zeigt DataFrame (wenn Ergebnisse existieren)
    """

    out = graph_chain.invoke({"query": question})

    sparql = out.get("sparql_query", "").strip()
    if sparql:
        print("🔍 Generierte SPARQL Query:")
        print("-" * 40)
        print(sparql)
        print("-" * 40 + "\n")
    else:
        print("⚠️ Keine SPARQL Query erzeugt.")
        return out

    # Ergebnis als Tabelle (über deine bestehende rdflib-Ausführung)
    if "sparql_select_raw" in globals():
        result = sparql_select_raw(sparql, max_rows=max_rows)

        if not result.get("ok"):
            print(f"❌ Fehler bei der Ausführung: {result.get('error')}")
            return out

        if result.get("row_count", 0) == 0:
            print("⚠️ Keine Ergebnisse gefunden.")
            return out

        df = pd.DataFrame(result["rows"])
        print(f"✅ {len(df)} Ergebnisse gefunden:")
        display(df)

    if show_llm_answer:
        print("\n🧠 LLM Antwort:")
        print(out.get("result", ""))

    return out

# Test:
run_graphsparql_chain("Welche POUs ruft 'HRL_SkillSet' auf?")


ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found '`'  (at char 0), (line:1, col:1)